# Core Authentication Utilities

> Simple, transparent authentication utilities for FastHTML applications

In [ ]:
#| default_exp auth

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import bcrypt
import secrets
from typing import Optional, Dict, Any
from functools import wraps

### Important Note on bcrypt

bcrypt has a maximum password length of 72 bytes. Passwords longer than this are silently truncated. If you need to support longer passwords, consider hashing the password with SHA-256 first, then passing the hash to bcrypt.

## Password Hashing

Launch Kit provides simple utilities for secure password hashing using bcrypt. These functions are transparent and composable - you can see exactly what they do and use them however you need.

In [ ]:
#| export
def hash_password(password: str) -> str:
    """Hash a password using bcrypt with a cost factor of 12.
    
    Args:
        password: Plain text password to hash
        
    Returns:
        Hashed password as a string for database storage
    """
    # Cost factor of 12 is recommended for 2024
    # This takes ~250ms on modern hardware
    salt = bcrypt.gensalt(rounds=12)
    hashed = bcrypt.hashpw(password.encode('utf-8'), salt)
    return hashed.decode('utf-8')

In [ ]:
#| export
def verify_password(password: str, hashed: str) -> bool:
    """Verify a password against a bcrypt hash.
    
    Uses timing-safe comparison to prevent timing attacks.
    
    Args:
        password: Plain text password to verify
        hashed: Bcrypt hash from the database
        
    Returns:
        True if password matches, False otherwise
    """
    try:
        return bcrypt.checkpw(
            password.encode('utf-8'), 
            hashed.encode('utf-8')
        )
    except (ValueError, TypeError):
        # Invalid hash format
        return False

### Example Usage

Here's how you use these password utilities in your FastHTML application:

In [ ]:
# Hash a password when user signs up
password = "secure_password123"
hashed = hash_password(password)
print(f"Hashed password: {hashed}")

# Verify password when user logs in
is_valid = verify_password(password, hashed)
print(f"Password valid: {is_valid}")

# Test with wrong password
wrong_password = "wrong_password"
is_valid = verify_password(wrong_password, hashed)
print(f"Wrong password valid: {is_valid}")

Hashed password: $2b$12$F0oaDZnVc3Ho967OIPj5XuCpnFn5lNZYWbf/of6t9uHFQtxW2Ckni
Password valid: True
Wrong password valid: False


In [ ]:
#| export
def get_user_from_session(sess) -> Optional[Dict[str, Any]]:
    """Extract user data from session.
    
    Args:
        sess: FastHTML session object
        
    Returns:
        User dictionary if authenticated, None otherwise
        
    Note:
        With FastHTML's built-in auth, you can also access auth status
        via the 'auth' parameter in route handlers.
    """
    if sess.get('auth') and 'user' in sess:
        return sess['user']
    return None

## Session Management

FastHTML provides a built-in `user_auth_before` function for authentication. When using Launch Kit's authentication utilities, you can leverage FastHTML's built-in authentication:

```python
from fasthtml.common import *

# Use FastHTML's built-in user_auth_before
beforeware = Beforeware(
    user_auth_before,  # Built-in function that redirects to /login
    skip=['/auth/login', '/auth/signup', '/static/.*']
)
```

The built-in `user_auth_before`:
- Checks `sess.get('auth')` for authentication status
- Sets `req.scope['auth']` which is automatically available to handlers
- Redirects to `/login` when not authenticated (303 redirect)

### Helper Functions

Launch Kit provides a convenience function for accessing user data from sessions:

## Authentication Tokens

For remember-me functionality and API authentication, we provide simple token utilities.

In [ ]:
#| export
def create_auth_token(user_id: int) -> str:
    """Create a secure authentication token for a user.
    
    Args:
        user_id: ID of the user to create token for
        
    Returns:
        Secure random token string
        
    Note:
        In production, you'd store this token in your database
        with an expiration date and associate it with the user.
    """
    # Generate a secure random token
    token = secrets.token_urlsafe(32)
    # In a real app, you'd store: token -> user_id mapping in database
    return f"{user_id}:{token}"

In [ ]:
#| export
def verify_auth_token(token: str) -> Optional[int]:
    """Verify an authentication token and return the user ID.
    
    Args:
        token: Authentication token to verify
        
    Returns:
        User ID if token is valid, None otherwise
        
    Note:
        In production, you'd look up this token in your database
        and check expiration.
    """
    try:
        # Simple parsing for demo - in production, look up in database
        parts = token.split(':', 1)
        if len(parts) == 2:
            user_id = int(parts[0])
            # In real app: verify token exists in DB and not expired
            return user_id
    except (ValueError, TypeError):
        pass
    return None

### Token Example

In [ ]:
# Create a token for user ID 123
token = create_auth_token(123)
print(f"Auth token: {token}")

# Verify the token
user_id = verify_auth_token(token)
print(f"Verified user ID: {user_id}")

# Test with invalid token
invalid_token = "invalid_token"
user_id = verify_auth_token(invalid_token)
print(f"Invalid token result: {user_id}")

Auth token: 123:aJXKHKstY8TWgyt94nym0_Cas2Fsb82f6hD8Wn15_Wk
Verified user ID: 123
Invalid token result: None


## Best Practices and Security Guidelines

### Password Security
- **Minimum Requirements**: Enforce at least 8 characters, ideally 12+
- **Complexity**: Require a mix of uppercase, lowercase, numbers, and special characters
- **Common Passwords**: Check against common password lists
- **Password History**: Prevent reuse of recent passwords
- **Account Lockout**: Implement after multiple failed attempts

### Session Security
- **Secure Cookies**: Always use `secure=True` and `httponly=True` in production
- **Session Timeout**: Implement idle and absolute timeouts
- **Session Rotation**: Regenerate session ID after login
- **CSRF Protection**: Use the CSRF middleware (coming in Phase 1)

### Token Security
- **Storage**: Never store tokens in plain text
- **Expiration**: Always set and enforce expiration times
- **Rotation**: Implement token rotation for long-lived sessions
- **Revocation**: Provide ability to revoke tokens

### General Security
- **HTTPS**: Always use HTTPS in production
- **Rate Limiting**: Implement on authentication endpoints
- **Logging**: Log authentication events (success/failure)
- **2FA**: Consider implementing two-factor authentication

## Live FastHTML Integration Example

Let's test these authentication utilities in a real FastHTML application to ensure they work correctly with sessions and beforeware.

In [ ]:
#| hide
# Create a simple in-memory user store for testing
users_db = {}
next_user_id = 1

def create_user(email, password):
    """Create a user in our test database"""
    global next_user_id
    user_id = next_user_id
    next_user_id += 1
    
    users_db[email] = {
        'id': user_id,
        'email': email,
        'password': hash_password(password)
    }
    return users_db[email]

def get_user_by_email(email):
    """Get user by email"""
    return users_db.get(email)

# Create a test user
test_user = create_user('test@example.com', 'password123')
print(f"Created test user: {test_user['email']} with ID {test_user['id']}")

Created test user: test@example.com with ID 1


### Manual Testing Instructions

To manually test the authentication flow with a live server, you can run the below cells in order which will run a FastHTML server inside the notebook using `JupyUvi`:

1. Visit http://0.0.0.0:8000 in your browser (or uncomment the `HTMX()` cell to view in nb)
2. Test the authentication flow
3. When done, stop the server:
```python
server.stop()
```

In [ ]:
#| eval: false
# Import FastHTML components
from fasthtml.common import *

# Create the FastHTML app with authentication beforeware
# Using FastHTML's built-in user_auth_before function
beforeware = Beforeware(
    user_auth_before,  # Built-in function that redirects to /login
    skip=[r'/auth/.*', r'/public/.*', '/']
)

app, rt = fast_app(before=beforeware)

# Public routes
@rt('/')
def get():
    return Div(
        H1('Authentication Test App'),
        P('This is the public home page'),
        A('Login', href='/auth/login'),
        ' | ',
        A('Protected Page', href='/protected')
    )

@rt('/auth/login')
def get():
    return Div(
        H1('Login'),
        Form(
            Input(name='email', type='email', placeholder='Email', value='test@example.com'),
            Input(name='password', type='password', placeholder='Password', value='password123'),
            Button('Login', type='submit'),
            method='post'
        )
    )

@rt('/auth/login')
def post(email: str, password: str, sess):
    user = get_user_by_email(email)
    
    if user and verify_password(password, user['password']):
        # Login successful - set auth in session
        sess['auth'] = True
        sess['user_id'] = user['id']
        sess['user'] = {'id': user['id'], 'email': user['email']}
        return Redirect('/protected')  # Redirect to protected page
    else:
        return Div(
            P('Invalid credentials', style='color: red'),
            A('Try again', href='/auth/login')
        )

@rt('/auth/logout')
def get(sess):
    sess.clear()
    return Div(
        P('Logged out successfully'),
        A('Go home', href='/')
    )

# Protected routes - auth is automatically available from req.scope['auth']
@rt('/protected')
def get(auth, sess):
    # auth parameter is automatically injected by FastHTML from req.scope['auth']
    if not auth:
        # This shouldn't happen with user_auth_before, but just in case
        return Redirect('/auth/login')
    
    user = sess.get('user', {})
    return Div(
        H1('Protected Page'),
        P(f"Welcome {user.get('email', 'User')}!"),
        P('This page requires authentication'),
        A('Logout', href='/auth/logout')
    )

print("FastHTML app created with built-in authentication")

### Running the Test Server

Now let's start the server and test the authentication flow. We'll use a specific port and ensure graceful shutdown.

In [ ]:
#| eval: false
# Create and start the server
# Note: This cell is for manual testing/demonstration only
from fasthtml.jupyter import JupyUvi, HTMX

server = JupyUvi(app)
print("Server started on http://0.0.0.0:8000")
print("\nTest URLs:")
print("- Home: http://0.0.0.0:8000/")
print("- Login: http://0.0.0.0:8000/auth/login")
print("- Protected: http://0.0.0.0:8000/protected")
print("\nTest credentials:")
print("- Email: test@example.com")
print("- Password: password123")

In [ ]:
# This doesn't display in the docs - uncomment and run it to see it in action
#HTMX()

### Testing the Authentication Flow Programmatically

Let's test the authentication flow using HTTP requests to verify everything works correctly.

In [ ]:
#| eval: false
# Test the authentication flow programmatically
# Note: This requires the server to be running from the previous cell
import httpx
import asyncio

# Wait a moment for the server to start
await asyncio.sleep(1)

# Create a client with cookie support
with httpx.Client(base_url="http://0.0.0.0:8000", follow_redirects=False) as client:
    print("Testing authentication flow...")
    
    # 1. Try to access protected page (should redirect to login)
    response = client.get("/protected")
    assert response.status_code == 303  # Redirect
    assert response.headers.get('location') == '/login'
    print("✓ Protected page redirects to login for unauthenticated user")
    
    # 2. Login with valid credentials
    login_data = {
        "email": "test@example.com",
        "password": "password123"
    }
    response = client.post("/auth/login", data=login_data)
    assert response.status_code == 303  # Redirect after successful login
    assert response.headers.get('location') == '/protected'
    print("✓ Login successful with redirect")
    
    # 3. Access protected page (should work now)
    response = client.get("/protected")
    assert response.status_code == 200
    assert "Welcome test@example.com" in response.text
    print("✓ Protected page accessible after login")
    
    # 4. Test invalid login
    bad_login_data = {
        "email": "test@example.com",
        "password": "wrong_password"
    }
    response = client.post("/auth/login", data=bad_login_data)
    assert response.status_code == 200  # Stay on login page
    assert "Invalid credentials" in response.text
    print("✓ Invalid credentials rejected")
    
    # 5. Logout
    response = client.get("/auth/logout")
    assert "Logged out successfully" in response.text
    print("✓ Logout successful")
    
    # 6. Try protected page again (should redirect)
    response = client.get("/protected")
    assert response.status_code == 303  # Redirect
    assert response.headers.get('location') == '/login'
    print("✓ Protected page redirects after logout")
    
print("\n✅ Full authentication flow working correctly with FastHTML's built-in auth!")

### Graceful Server Shutdown

It's important to stop the server properly to avoid leaving threads running.

In [ ]:
#| eval: false
# Stop the server gracefully
# Note: Always run this after testing to clean up
print("Stopping server...")
server.stop()
print("✓ Server stopped gracefully")
print("\n🎉 All tests passed! Authentication utilities are working correctly with FastHTML.")

## Testing

In [ ]:
#| hide
# Test password hashing
test_password = "test_password_123!"
hashed = hash_password(test_password)

# Ensure hash is different from password
assert hashed != test_password
assert len(hashed) == 60  # bcrypt hashes are always 60 chars
assert hashed.startswith('$2b$')  # bcrypt prefix

# Test password verification
assert verify_password(test_password, hashed) == True
assert verify_password("wrong_password", hashed) == False

# Test with empty password
empty_hash = hash_password("")
assert verify_password("", empty_hash) == True
assert verify_password("not_empty", empty_hash) == False

# Test invalid hash handling
assert verify_password("any_password", "invalid_hash") == False
assert verify_password("any_password", "") == False

print("✓ Password hashing tests passed")

✓ Password hashing tests passed


In [ ]:
#| hide
# Test token creation and verification
user_id = 42
token = create_auth_token(user_id)

# Token should contain user ID and random part
assert ':' in token
assert token.startswith(f"{user_id}:")

# Verify token returns correct user ID
verified_id = verify_auth_token(token)
assert verified_id == user_id

# Test invalid tokens
assert verify_auth_token("invalid") is None
assert verify_auth_token("") is None
assert verify_auth_token("no_colon") is None
assert verify_auth_token("not_a_number:token") is None

print("✓ Token tests passed")

✓ Token tests passed


In [ ]:
#| hide
# Test session utilities - removed tests for our custom user_auth_before
# FastHTML's built-in user_auth_before handles authentication redirects

# We still keep get_user_from_session for convenience
mock_sess = {}

# Test get_user_from_session with no auth
assert get_user_from_session(mock_sess) is None

# Test with auth but no user
mock_sess['auth'] = True
assert get_user_from_session(mock_sess) is None

# Test with auth and user
mock_sess['user'] = {'id': 1, 'email': 'test@example.com'}
user = get_user_from_session(mock_sess)
assert user is not None
assert user['id'] == 1
assert user['email'] == 'test@example.com'

print("✓ Session utility tests passed")

In [ ]:
#| hide
# Test concurrent access patterns (simulated)
print("Testing thread safety simulation...")

# Test multiple password operations
passwords = ["pass1", "pass2", "pass3", "pass4", "pass5"]
hashes = [hash_password(p) for p in passwords]

# Verify all hashes are unique
assert len(set(hashes)) == len(hashes), "Hashes should be unique"

# Verify all passwords match their respective hashes
for pwd, hsh in zip(passwords, hashes):
    assert verify_password(pwd, hsh) == True
    # And don't match others
    for other_hash in hashes:
        if other_hash != hsh:
            assert verify_password(pwd, other_hash) == False

print("✓ Concurrent access simulation passed")

Testing thread safety simulation...
✓ Concurrent access simulation passed


In [ ]:
#| hide
# Test edge cases for password hashing
print("Testing edge cases...")

# Test with very long password (bcrypt truncates at 72 bytes)
long_password = "a" * 100  # Changed from 1000 to avoid bcrypt truncation
long_hash = hash_password(long_password)
assert verify_password(long_password, long_hash) == True
# Test that a different password doesn't match
assert verify_password("b" * 100, long_hash) == False

# Test with special characters
special_password = "!@#$%^&*()_+-=[]{}|;':\",./<>?`~"
special_hash = hash_password(special_password)
assert verify_password(special_password, special_hash) == True

# Test with unicode characters
unicode_password = "пароль密码パスワード🔐"
unicode_hash = hash_password(unicode_password)
assert verify_password(unicode_password, unicode_hash) == True

# Test with None/null handling (should raise AttributeError)
try:
    hash_password(None)
    assert False, "Should have raised AttributeError"
except AttributeError:
    pass

# Test corrupted hash
assert verify_password("test", "$2b$12$corrupted") == False
assert verify_password("test", "not_a_bcrypt_hash") == False

print("✓ Edge case tests passed")

Testing edge cases...
✓ Edge case tests passed


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

## Performance Benchmarks

Understanding the performance characteristics of authentication operations is crucial for capacity planning.

In [ ]:
#| hide
# Comprehensive performance benchmarks
import time
import statistics

print("Running performance benchmarks...")

# Benchmark password hashing with different cost factors
def benchmark_cost_factor(cost):
    times = []
    for _ in range(3):
        start = time.time()
        salt = bcrypt.gensalt(rounds=cost)
        bcrypt.hashpw(b"benchmark_password", salt)
        times.append(time.time() - start)
    return statistics.mean(times)

print("\nPassword hashing performance by cost factor:")
for cost in [10, 11, 12, 13, 14]:
    avg_time = benchmark_cost_factor(cost)
    print(f"  Cost {cost}: {avg_time:.3f}s (recommended: {'Yes' if 0.1 <= avg_time <= 0.5 else 'No'})")

# Benchmark verification
print("\nPassword verification performance:")
test_hash = hash_password("benchmark_password")
verify_times = []
for _ in range(10):
    start = time.time()
    verify_password("benchmark_password", test_hash)
    verify_times.append(time.time() - start)

avg_verify = statistics.mean(verify_times)
print(f"  Average verification time: {avg_verify:.3f}s")
print(f"  Verifications per second: {1/avg_verify:.0f}")

# Token operations benchmark
print("\nToken operations performance:")
token_create_times = []
token_verify_times = []

for i in range(100):
    start = time.time()
    token = create_auth_token(i)
    token_create_times.append(time.time() - start)
    
    start = time.time()
    verify_auth_token(token)
    token_verify_times.append(time.time() - start)

print(f"  Token creation: {statistics.mean(token_create_times)*1000:.3f}ms")
print(f"  Token verification: {statistics.mean(token_verify_times)*1000:.3f}ms")
print(f"  Operations per second: {1/statistics.mean(token_create_times):.0f}")

print("\n✓ Performance benchmarks completed")

Running performance benchmarks...

Password hashing performance by cost factor:
  Cost 10: 0.043s (recommended: No)
  Cost 11: 0.085s (recommended: No)
  Cost 12: 0.191s (recommended: Yes)
  Cost 13: 0.360s (recommended: Yes)
  Cost 14: 0.670s (recommended: No)

Password verification performance:
  Average verification time: 0.168s
  Verifications per second: 6

Token operations performance:
  Token creation: 0.002ms
  Token verification: 0.000ms
  Operations per second: 436452

✓ Performance benchmarks completed
